In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sps

In [2]:
data = pd.read_csv('cps99_ps3.csv')
data

,ahe,female,ne,midwest,south,west,race,yrseduc,ba,hsdipl,age
0,10.754048,1,1,0,0,0,1,18,1,1,49
1,10.341881,0,1,0,0,0,1,16,1,1,28
2,18.181818,0,1,0,0,0,1,12,0,1,45
3,20.879122,1,1,0,0,0,1,12,0,1,41
4,17.948717,0,1,0,0,0,1,18,1,1,54
...,...,...,...,...,...,...,...,...,...,...,...
37805,41.208790,0,0,0,0,1,4,19,1,1,51
37806,21.634615,0,0,0,0,1,4,16,1,1,44
37807,8.636364,1,0,0,0,1,4,14,0,1,44
37808,10.544871,0,0,0,0,1,4,13,0,1,33


In [6]:
y = np.log(data['ahe'])
x = np.array([data.yrseduc, data.age, np.power(data.age, 2), data.female, data.hsdipl]).T
x = np.column_stack((np.ones(x.shape[0]), x))
beta_hat = np.linalg.inv(x.T @ x) @ x.T @ y

print('a)')
print(f"Коэффициент при age = {beta_hat[2]}, при age^2 = {beta_hat[3]}.")
print("Возраст оказывает примерно 4% влияния на зарплату, однако большой возраст (пожилой человек) оказывает небольшой влияение на зарплату, уменьшая её.")

a)
Коэффициент при age = 0.04495285248349277, при age^2 = -0.0004409892166350506.
Возраст оказывает примерно 4% влияния на зарплату, однако большой возраст (пожилой человек) оказывает небольшой влияение на зарплату, уменьшая её.


In [23]:
# С помощью F-теста проверим, является ли коэффициент при age^2 значимым.
# H0: коэффициент при age^2 равен 0, то есть связь между ln(ahe) и возрастом линейна.
# Ha: коэффициент при age^2 не равен 0, то есть связь между ln(ahe) и возрастом является полиномом второй степени.

alpha = 0.05

x_lin = np.array([data.yrseduc, data.age, data.female, data.hsdipl]).T
x_lin = np.column_stack((np.ones(x_lin.shape[0]), x_lin))

beta_hat_lin = np.linalg.inv(x_lin.T @ x_lin) @ x_lin.T @ y

rss_2polinom = np.sum((y - x @ beta_hat)**2)
rss_lin = np.sum((y - x_lin @ beta_hat_lin)**2)

k = 1
n = x.shape[0]
j = 5
f_stat = ((rss_lin - rss_2polinom)/k)/(rss_2polinom/(n - k - j))

p_value = 1 - sps.f.cdf(f_stat, k, n - k - j)

print('b)')
if p_value > alpha:
    print("Не отвергаем нулевую гипотезу. => связь между ln(ahe) и возрастом линейна")
else:
    print("Отвергаем нулевую гипотезу в пользу альтернативы. => связь между ln(ahe) и возрастом является полиномом второй степени.")

b)
Отвергаем нулевую гипотезу в пользу альтернативы. => связь между ln(ahe) и возрастом является полиномом второй степени.


In [22]:
# Сравним оценку коэффициента при female в модели с age^2 и без:

delta = abs(beta_hat[4] - beta_hat_lin[3])

print('c)')
print("Разность между оценками коэффициента при female в модели с age^2 и без = ", delta)
print("Так как разность очень мала, можно сделать вывод, что включение age^2 не изменяет оценку разрыва доходов.")

c)
Разность между оценками коэффициента при female в модели с age^2 и без =  0.0006433343735996466
Так как разность очень мала, можно сделать вывод, что включение age^2 не изменяет оценку разрыва доходов


In [31]:
# С помощью F-теста проверим, является ли коэффициент при age^3 значимым.
# H0: коэффициент при age^3 равен 0, то есть связь между ln(ahe) и возрастом является полиномом второй степени.
# Ha: коэффициент при age^3 не равен 0, то есть связь между ln(ahe) и возрастом является полиномом третьей степени.

x_3polinom = np.array([data.yrseduc, data.age, np.power(data.age, 2), np.power(data.age, 3), data.female, data.hsdipl]).T
x_3polinom = np.column_stack((np.ones(x_3polinom.shape[0]), x_3polinom))
beta_hat_3polinom = np.linalg.inv(x_3polinom.T @ x_3polinom) @ x_3polinom.T @ y

rss_3polinom = np.sum((y - x_3polinom @ beta_hat_3polinom)**2)

k = 1
n = x.shape[0]
j = 6
f_stat = ((rss_2polinom - rss_3polinom)/k)/(rss_3polinom/(n - k - j))

p_value = 1 - sps.f.cdf(f_stat, k, n - k - j)

print('d)')
if p_value > alpha:
    print("Не отвергаем нулевую гипотезу. => связь между ln(ahe) и возрастом является полиномом второй степени.")
else:
    print("Отвергаем нулевую гипотезу в пользу альтернативы. => связь между ln(ahe) и возрастом является полиномом третьей степени.")

d)
Отвергаем нулевую гипотезу в пользу альтернативы. => связь между ln(ahe) и возрастом является полиномом третьей степени.


In [34]:
# С помощью F-теста проверим, являются ли коэффициенты при age^2 и age^3 значимыми.
# H0: коэффициенты при age^2 и age^3 равны 0, то есть связь между ln(ahe) и возрастом является линейной.
# Ha: коэффициент при age^2 или age^3 не равен 0, то есть связь между ln(ahe) и возрастом является полиномом степени не выше 3.

k = 2
n = x.shape[0]
j = 5
f_stat = ((rss_lin - rss_3polinom)/k)/(rss_3polinom/(n - k - j))

p_value = 1 - sps.f.cdf(f_stat, k, n - k - j)

print('e)')
if p_value > alpha:
    print("Не отвергаем нулевую гипотезу. => связь между ln(ahe) и возрастом линейна.")
else:
    print("Отвергаем нулевую гипотезу в пользу альтернативы. => связь между ln(ahe) и возрастом является полиномом степени не выше 3.")

print("Так как тесты отвергли линейную связть в пользу полинома степени не выше 3, а так же отвергли свзять в виде полинами степени 2 в пользу полинома степени 3,")
print("то я порекомендую использовать спецификацию с полиномом 3 степени, то есть добавить в модель age, age^2 и age^3.")

e)
Отвергаем нулевую гипотезу в пользу альтернативы. => связь между ln(ahe) и возрастом является полиномом степени не выше 3.
Так как тесты отвергли линейную связть в пользу полинома степени не выше 3, а так же отвергли свзять в виде полинами степени 2 в пользу полинома степени 3,
то я порекомендую использовать спецификацию с полиномом 3 степени, то есть добавить в модель age, age^2 и age^3.
